In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
!pip install -r requirements.txt
dbutils.library.restartPython()

In [0]:
from constants import * 
import networkx as nx
import pyvis

from src.helper_functions import *
from src.llama_index_graph_rag_extractor import GraphRAGExtractor
from src.graph_rag_store import GraphRAGStore

2025-06-26 22:33:16.994407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750977197.013531    1784 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750977197.019859    1784 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-26 22:33:17.040775: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [0]:
# Manage environment variables required to attach LLM service in this context
exec(open('azure_envars.py').read())

In [0]:
from main import create_graph_index
save_idx_path = os.path.join(os.getcwd(), "outputs/palantir_8k_10k_graph")
save_kg_path = os.path.join(os.getcwd(), "outputs/8k_10k_kg.html")

index = create_graph_index(docs_dir= "data",
                           # num_nodes = 1,
                           save_index = True,
                           save_kg = False,
                           index_path = save_idx_path,
                           kg_path = save_kg_path
                        )
# index.property_graph_store.build_communities()

In [0]:
from src.helper_functions import kg_relations_to_df
kg_relations_to_df(index, save_as_csv = True, filename = "outputs/8k_10k_kg_relations.csv")

,subject,predicate,object,filename,company,document_type,timestamp,period,relationship_description
0,Palantir Technologies Inc.,Issues,Class A Common Stock,2024 FY.pdf,Palantir Technologies,10K report,2024,Yearly,Palantir Technologies Inc. issues Class A Comm...
1,Class A Common Stock,Traded under symbol,PLTR,2024 FY.pdf,Palantir Technologies,10K report,2024,Yearly,Palantir's Class A Common Stock is traded unde...
2,PLTR,Listed on,The Nasdaq Stock Market LLC,2024 FY.pdf,Palantir Technologies,10K report,2024,Yearly,The PLTR ticker is listed on The Nasdaq Stock ...
3,Palantir Technologies Inc.,Files,10K report,2024 FY.pdf,Palantir Technologies,10K report,2024,Yearly,Palantir Technologies Inc. files an annual 10K...
4,10K report,Submitted to,United States Securities and Exchange Commission,2024 FY.pdf,Palantir Technologies,10K report,2024,Yearly,The 10K report is submitted to the SEC as requ...
...,...,...,...,...,...,...,...,...,...
4994,Twelfth Amendment,Amends,Credit Agreement,31_03_2022.pdf,Palantir Technologies,8K report,31-03-2022,,The Twelfth Amendment modifies the terms of th...
4995,Exhibit 10.1,Contains,Twelfth Amendment,31_03_2022.pdf,Palantir Technologies,8K report,31-03-2022,,Exhibit 10.1 includes the full text of the Twe...
4996,Palantir Technologies,No Outstanding Debt,Outstanding Amounts,31_03_2022.pdf,Palantir Technologies,8K report,31-03-2022,,"As of the report date, Palantir Technologies h..."
4997,Alexander C. Karp,Executive Officer,Palantir Technologies,31_03_2022.pdf,Palantir Technologies,8K report,31-03-2022,,Alexander C. Karp is the Chief Executive Offic...


### Example Use case

In [0]:
query_str = "What was Palantir's revenue in 2022?"

nodes = index.as_retriever(
            similarity_top_k= 8
        ).retrieve(query_str)
nodes

[NodeWithScore(node=TextNode(id_='10940b15-1320-4346-9b28-5c4229112bb8', embedding=None, metadata={'company': 'Palantir Technologies', 'document_type': 'Financial report', 'year': '2022', 'period': 'FY'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2022 FY', node_type='4', metadata={'company': 'Palantir Technologies', 'document_type': 'Financial report', 'year': '2022', 'period': 'FY'}, hash='0df669ed3f4792903f9dd2eec2a8d3218fcc6b2b19f26987e255d6f686545d74'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='aa5da375-da1d-4ec6-b91f-c8b8e803366b', node_type='1', metadata={'company': 'Palantir Technologies', 'document_type': 'Financial report', 'year': '2022', 'period': 'FY'}, hash='aaea765350213aaa14cd7a06c06d7c9a15547f29661bb43621b79e098caa6bcc'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='20ee1a95-4dd0-4bec-b521-ad203cb0594c', node_type='1', metadata={}, hash='f5564e6c8ac2a

[Trace(request_id=tr-e719d79487a5415aa31a3d3c3c5d3706), Trace(request_id=tr-5396ada6523e4d89a5a6fdeb68540ea5)]

### ✅ TODO List:


#### 📦 Explore N-tuple Abstraction
- Define a flexible `n`-tuple data structure for representing knowledge graph entities and relationships 
- Extend to support:
  - Higher-order relationships (e.g., quads or n-tuples).
  - Temporal and contextual metadata.
- Provide transformation utilities between `n`-tuples and NetworkX / Neo4j graph formats.
